In [1]:
%pip install transformers==4.20.1 -q
%pip freeze | grep transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 74.2 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 111.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 23.9 MB/s eta 0:00:00
transformers==4.20.1


In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('gpt2-xl')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')

Downloading:   0%|          | 0.00/689 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.99G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [3]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1600)
    (wpe): Embedding(1024, 1600)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout)

In [5]:
force_words_ids = tokenizer(['sun', 'blade', 'heart', 'task'], add_prefix_space=True, add_special_tokens=False)['input_ids']

outputs = model.generate(
    force_words_ids=force_words_ids,
    num_beams=5,
    remove_invalid_values=True,
    early_stopping=True,
    no_repeat_ngram_size=2,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


A new task force is heartening the sunnier side of the blade.

The


In [6]:
import requests

def get_kanji(id):
    url = f"https://api.robanohashi.org/kanji/{id}"
    response = requests.get(url)
    return response.json()

def get_mnemonic_by_kanji(kanji_id):
    url = f"https://api.robanohashi.org/subject/{kanji_id}/meaning_mnemonics"
    response = requests.get(url)
    return response.json()

In [7]:
kanji_id_range = range(450, 2450)

import random
kanji_ids = random.sample(kanji_id_range, 200)

In [8]:
import pandas as pd
import tqdm.auto as tqdm

bar = tqdm.tqdm(kanji_ids)

df = pd.DataFrame(columns=['kanji_id', 'component_meanings', 'wanikani_mnemonic', 'kanji_meaning'])


for id in kanji_ids:
    kanji = get_kanji(id)
    mnemonics = get_mnemonic_by_kanji(id)
    kanji_meaning = [m['meaning'] for m in kanji['meanings'] if m['primary']][0]
    component_meanings = [c['meanings'][0] for c in kanji['component_subjects']]
    mnemonic_text = [m['text'] for m in mnemonics['items'] if m['user_id'] == 'wanikani'][0]
    df = pd.concat([pd.DataFrame([[kanji['id'],
                    ','.join(component_meanings),mnemonic_text,kanji_meaning]], columns=df.columns), df], ignore_index=True)
    bar.update(1)

df

  0%|          | 0/200 [00:00<?, ?it/s]

,kanji_id,component_meanings,wanikani_mnemonic,kanji_meaning
0,1885,"Woman,Catapult",A <radical>woman</radical> loves her <radical>...,Recreation
1,1053,Paragraph,The <radical>paragraph</radical> radical and t...,Paragraph
2,1507,"Valley,Lack",In the <radical>valley</radical> you feel a <r...,Want
3,1127,"Half,Knife","""I'll cut that baby in <radical>half</radical>...",Judge
4,2041,"Moon,Power",The <radical>moon</radical> has a lot of <radi...,Armpit
...,...,...,...,...
195,1980,"Crab,Lack",A <radical>crab</radical> will try to hide wha...,Deceit
196,2335,"Leader,Drunkard",If the <radical>leader</radical> of your gang ...,Fell
197,2440,"Flowers,Oversee",<radical>Flowers</radical> that you <radical>o...,Indigo
198,611,"Drop,Mustache",A <radical>drop</radical> hits your <radical>m...,Yonder


In [9]:
mean_tokens = df.apply(lambda x: len(tokenizer(x['wanikani_mnemonic'], add_special_tokens=False).input_ids), axis=1).mean()

print(f"Mean tokens: {mean_tokens}")

Mean tokens: 95.525


In [10]:
def generate_menmonic(force_words):
    force_words_ids = tokenizer(force_words, add_prefix_space=True, add_special_tokens=False)['input_ids']
    outputs = model.generate(
        force_words_ids=force_words_ids,
        num_beams=10,
        remove_invalid_values=True,
        max_length=70,
        early_stopping=True,
        no_repeat_ngram_size=1,
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [11]:
df['component_meanings'] = df['component_meanings'].apply(lambda x: x.lower())

In [12]:
df['kanji_meaning'] = df['kanji_meaning'].apply(lambda x: x.lower())

In [14]:
df[df['component_meanings'].str.contains('dry')]

,kanji_id,component_meanings,wanikani_mnemonic,kanji_meaning
20,1014,dry,The <radical>dry</radical> radical and the <ka...,dry
128,503,"slide,dry",The <radical>slide</radical> is <radical>dry</...,noon
164,535,"dry,horns",If you want to <radical>dry</radical> your <ra...,flat


In [29]:
import re
exp = r"The <radical>.+</radical> radical and the <kanji>.+</kanji> kanji are the same"

# remove mnemonics that contain the same radical and kanji

def filter_mnemonic(mnemonic):
    return re.search(exp, mnemonic) is not None

mask = df['wanikani_mnemonic'].apply(filter_mnemonic)
df = df[~mask].reset_index(drop=True)


In [31]:
len(df)

194

In [32]:
idx = 20

row = df.iloc[idx]
component_meanings = row['component_meanings'].split(',')
kanji_meaning = row['kanji_meaning']
force_words = component_meanings + [kanji_meaning]

print(f"Force words: {force_words}")
generate_menmonic(force_words)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Force words: ['thread', 'hill', 'netting']


"This is a thread netting for all of you hillbillies out there.\n\n\nI'm not sure if this has ever been done before, but I thought it would be fun to see what we could come up with and maybe even get some ideas from other people who are more skilled in the art than me (or at least smarter)."

In [38]:
import tqdm.auto as tqdm
bar = tqdm.tqdm(range(len(df)))

def fill_mnemonic(row):
    component_meanings = row['component_meanings'].split(',')
    kanji_meaning = row['kanji_meaning']
    force_words = component_meanings + [kanji_meaning]
    bar.update(1)
    return generate_menmonic(force_words)

df['gpt2xl_mnemonic'] = df.apply(fill_mnemonic, axis=1)

  0%|          | 0/194 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

In [39]:
df.head()

,kanji_id,component_meanings,wanikani_mnemonic,kanji_meaning,gpt2xl_mnemonic
0,1885,"woman,catapult",A <radical>woman</radical> loves her <radical>...,recreation,This is a rush transcript. Copy may not be in ...
1,1053,paragraph,The <radical>paragraph</radical> radical and t...,paragraph,"It's been a long time coming, but the first pa..."
2,1507,"valley,lack",In the <radical>valley</radical> you feel a <r...,want,I want to start off by saying that I am a vall...
3,1127,"half,knife","""I'll cut that baby in <radical>half</radical>...",judge,This is the first half of a two-part series. C...
4,2041,"moon,power",The <radical>moon</radical> has a lot of <radi...,armpit,"It's been armpit time for a while now, but moo..."


In [40]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)

In [41]:
dataset.to_json('dataset.jsonl', orient='records', lines=True)

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

157873

In [42]:
%pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 1.7 MB/s eta 0:00:00a 0:00:01


In [45]:
from dotenv import load_dotenv

load_dotenv()

True

In [47]:
import os 

openai_api_key = os.getenv('OPENAI_API_KEY')

In [48]:
import openai

openai.api_key = openai_api_key

In [54]:
def gen_gpt3_mnemonic(force_words):
    res = openai.Completion.create(
        model="text-davinci-003",
        prompt=f"Write a short paragraph that includes the following words: {','.join(force_words)}\n",
        temperature=0.5,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    return res['choices'][0]['text'].replace('\n','')

In [65]:
import re
exp = r"The <radical>.+</radical> radical and the <kanji>.+</kanji> kanji"

# remove mnemonics that contain the same radical and kanji

def filter_mnemonic(mnemonic):
    return re.search(exp, mnemonic) is not None

mask = df['wanikani_mnemonic'].apply(filter_mnemonic)
df = df[~mask].reset_index(drop=True)

In [66]:
len(df)

190

In [72]:
import tqdm.auto as tqdm
bar = tqdm.tqdm(range(len(df)))

def fill_mnemonic(row):
    component_meanings = row['component_meanings'].split(',')
    kanji_meaning = row['kanji_meaning']
    force_words = component_meanings + [kanji_meaning]
    bar.update(1)
    return gen_gpt3_mnemonic(force_words)

df['gpt3_paragraph'] = df.apply(fill_mnemonic, axis=1)

  0%|          | 0/190 [00:00<?, ?it/s]

In [73]:
dataset = Dataset.from_pandas(df)

In [74]:
dataset.to_json('dataset.jsonl', orient='records', lines=True)

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

199457

In [75]:
df.head()

,kanji_id,component_meanings,wanikani_mnemonic,kanji_meaning,gpt2xl_mnemonic,gpt3_paragraph
0,1885,"woman,catapult",A <radical>woman</radical> loves her <radical>...,recreation,This is a rush transcript. Copy may not be in ...,A woman was using a catapult for recreation in...
1,1507,"valley,lack",In the <radical>valley</radical> you feel a <r...,want,I want to start off by saying that I am a vall...,The valley was filled with a lack of resources...
2,1127,"half,knife","""I'll cut that baby in <radical>half</radical>...",judge,This is the first half of a two-part series. C...,I was only half paying attention when the judg...
3,2041,"moon,power",The <radical>moon</radical> has a lot of <radi...,armpit,"It's been armpit time for a while now, but moo...",The moon has a mysterious power that can be fe...
4,1758,"gold,guard",If you have <radical>gold</radical> you also n...,sharp,This is a rush transcript. Copy may not be in ...,"The guard was dressed in a regal gold uniform,..."


In [85]:
def gen_gpt3_mnemonic(component_meanings, kanji_meaning):
    res = openai.Completion.create(
        model="text-davinci-003",
        prompt=f"Write a short paragraph long mnemonic story that is based on and must include the words: <{','.join(component_meanings)},{kanji_meaning}>\n",
        temperature=0.5,
        max_tokens=120,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    return res['choices'][0]['text'].replace('\n','')

In [86]:
import tqdm.auto as tqdm
bar = tqdm.tqdm(range(len(df)))

def fill_mnemonic(row):
    component_meanings = row['component_meanings'].split(',')
    kanji_meaning = row['kanji_meaning']
    bar.update(1)
    return gen_gpt3_mnemonic(component_meanings, kanji_meaning)

df['gpt3_mnemonic'] = df.apply(fill_mnemonic, axis=1)

  0%|          | 0/190 [00:00<?, ?it/s]

In [87]:
dataset = Dataset.from_pandas(df)

dataset.to_json('dataset.jsonl', orient='records', lines=True)

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

273680